In [1]:
!pip install -qU split-folders

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from tensorflow.keras.utils import to_categorical
import numpy as np
import splitfolders

In [3]:
train_datagen = ImageDataGenerator(
rescale=1./255,
rotation_range=40,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True,
fill_mode='nearest'
)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
!curl -L -o archive.zip https://www.kaggle.com/api/v1/datasets/download/jessicali9530/celeba-dataset
!unzip -qq -o archive.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1361M  100 1361M    0     0  21.9M      0  0:01:01  0:01:01 --:--:-- 22.7M
Archive:  archive.zip
replace img_align_celeba/img_align_celeba/000001.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [5]:
path="/content/img_align_celeba"
splitfolders.ratio(path,seed=1337, output="/content/celeba-split", ratio=(0.6, 0.2, 0.2))

Copying files: 202599 files [01:39, 2034.44 files/s]


In [6]:
train_generator = train_datagen.flow_from_directory(
'/content/celeba-split/train',
target_size=(218, 178),
batch_size=32,
class_mode='binary'
)
val_generator = val_datagen.flow_from_directory(
'/content/celeba-split/val',
target_size=(218, 178),
batch_size=32,
class_mode='binary'

)
test_generator = test_datagen.flow_from_directory(
'/content/celeba-split/test',
target_size=(218, 178),
batch_size=32,
class_mode='binary'
)

Found 121559 images belonging to 1 classes.
Found 40519 images belonging to 1 classes.
Found 40521 images belonging to 1 classes.


In [10]:
model = models.Sequential()
model.add(layers.Conv2D(64, (5, 5), strides=2, padding='same', activation='relu', input_shape=(218, 178, 3)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(128, (5, 5), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.2))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.2))
model.add(layers.Dense(40))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy']
)

In [ ]:
model.fit(
train_generator,
epochs=10,
validation_data=val_generator,
verbose=1
)

Epoch 1/10
 283/3799 ━━━━━━━━━━━━━━━━━━━━ 18:42 319ms/step - accuracy: 0.5879 - loss: 2.0270